<a href="https://colab.research.google.com/github/zibro011/DataScience-Lab7/blob/main/ZibrowskiLab7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**INET 4061 Lab 7** <br>
**Adelaide Zibrowski** <br>
**October 25, 2021**


#Overview
For this lab, we are analyzing a data set that contains information regarding different bottles of wine. The dataset consists of 178 bottles of wine from 3 different regions of Italy. Using the available data, it is our responsibility to figure out which region of Italy each bottle of wine originated from. We will do so using basic Random Forest, k-Nearest Neighbors, SVC, and Logistic Regression models. Then, we will apply the Bagging, Boosting, and Stacking ensemble learning techniques to those four models. Each ensemble learning technique will be defined in its corresponding section of the lab. While the business problem is to assign each bottle of wine to its region, we will mainly be using this lab to evaluate the accuracy of different models using different learning techniques.


#Data
This dataset is obtained from https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html.

In [ ]:
pip install mlxtend


In [ ]:
pip install xgboost

In [ ]:
#conda install -c conda-forge xgboost
#conda install -c conda-forge mlxtend
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

from sklearn.datasets import load_wine
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import model_selection
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from mlxtend.classifier import StackingClassifier
from mlxtend.classifier import StackingCVClassifier
from mlxtend.classifier import EnsembleVoteClassifier

from xgboost import XGBClassifier


/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


In [ ]:
wine = load_wine()
# Convert to Pandas Dataframe
wine_dataframe = pd.DataFrame(data = np.c_[wine['data'], wine['target']], 
                             columns = wine['feature_names'] + ['target'])
print(wine.DESCR)
wine_dataframe.round(4).describe()


.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            1

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000
mean,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258,0.938202
std,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474,0.775035
min,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000,0.000000
25%,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000,0.000000
50%,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000,1.000000
75%,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000,2.000000
max,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000,2.000000


#Exploratory Data Analysis (EDA)

##Understand the Data

In [ ]:
wine_dataframe.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0.0


In [ ]:
wine_dataframe.tail()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.7,0.64,1.74,740.0,2.0
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.3,0.70,1.56,750.0,2.0
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.2,0.59,1.56,835.0,2.0
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.3,0.60,1.62,840.0,2.0
177,14.13,4.10,2.74,24.5,96.0,2.05,0.76,0.56,1.35,9.2,0.61,1.60,560.0,2.0


In [ ]:
wine_dataframe.shape

(178, 14)

In [ ]:
wine_dataframe.nunique()

alcohol                         126
malic_acid                      133
ash                              79
alcalinity_of_ash                63
magnesium                        53
total_phenols                    97
flavanoids                      132
nonflavanoid_phenols             39
proanthocyanins                 101
color_intensity                 132
hue                              78
od280/od315_of_diluted_wines    122
proline                         121
target                            3
dtype: int64

In [ ]:
wine_dataframe['target'].unique()

array([0., 1., 2.])

In [ ]:
class1 = len(wine_dataframe[wine_dataframe['target']== 0.0])
class2 = len(wine_dataframe[wine_dataframe['target']== 1.0])
class3 = len(wine_dataframe[wine_dataframe['target']== 2.0])

print("Class 1 Occurances: " + str(class1))
print("Class 2 Occurances: " + str(class2))
print("Class 3 Occurances: " + str(class3))

Class 1 Occurances: 59
Class 2 Occurances: 71
Class 3 Occurances: 48


From the code above, we can see that we have a dataset with 178 rows and 14 columns. The 14 columns are: alcohol, malic_acid, ash, alcalinity_of_ash, magnesium, total_phenols, flavanoids, nonflavanoid_phenols, proanthocyanins, color_intensity, hue, od280/od315_of_diluted_wines, proline, and target. 
The target variable has 3 possible values: 0, 1, and 2 which each correspond to a different region of Italy. From here on out, I will refer to the regions as 1, 2, and 3 respectively because that is what 0, 1, and 2 correspond to. There are 59 wines from region 1, 71 wines from region 2, and 48 wines from region 3. While this is somewhat unbalanced, I have decided that it is balanced enough to still work fine. However, if I have trouble fitting my models later, I may have to balance the data. I also printed the metrics in terms of mean, max and min in the Data section. However, these numbers are difficult to interpret without a visualization because we are using categorical data.

##Clean the Data

In [ ]:
wine_dataframe.isnull().sum()

alcohol                         0
malic_acid                      0
ash                             0
alcalinity_of_ash               0
magnesium                       0
total_phenols                   0
flavanoids                      0
nonflavanoid_phenols            0
proanthocyanins                 0
color_intensity                 0
hue                             0
od280/od315_of_diluted_wines    0
proline                         0
target                          0
dtype: int64

In [ ]:
#drop any rows that contain an outlier in any column
#here I define an outlier as a value that is more than 3 standard deviations away from the mean
col = wine_dataframe.iloc[:,1:]
wine_dataframe[(np.abs(stats.zscore(col)) < 3)]
wine_dataframe.shape

(178, 14)

We can see that the dataset does not contain any null values or outliers. Similar to Lab 6, we cannot have any outliers because of the data's categorical nature. It is still useful to take a look at our maximum and minimum values to make sure no categories are too spread out. For example, if one category had a minimum of 5 and a maximum of 5000, we could reasonably assume that there were value(s) in the dataset that were incorrect and would need to be discarded. Our proline variable seems to have an oddly large range between maximum and minimum values. Since I did not know what proline meant, I went ahead and did a quick search and found that proline values can be anywhere between 0 and 3400, so our range is reasonable here (https://ojs.openagrar.de/index.php/VITIS/article/view/7459#:~:text=Th%20e%20ranges%20found%20in,3400%20mg%2Fl%20of%20proline).

#Models
In the creation and analysis of my models, I used information from https://medium.com/@saugata.paul1010/ensemble-learning-bagging-boosting-stacking-and-cascading-classifiers-in-machine-learning-9c66cb271674.

##Base Learners
Here we create and assess the accuracy of our base models with no ensemble classifiers, so we have a basis to compare to.

In [ ]:
X = wine_dataframe.drop("target", 1)
y = wine_dataframe.target

RANDOM_SEED = 4061

#Base Learners
rf_clf = RandomForestClassifier(n_estimators=10, random_state=RANDOM_SEED)
knn_clf = KNeighborsClassifier(n_neighbors=2)
svc_clf = SVC(C=10000.0, kernel='rbf', random_state=RANDOM_SEED)
lr_clf = LogisticRegression(C=20000, penalty='l2', random_state=RANDOM_SEED)

classifier_array = [rf_clf, knn_clf, svc_clf, lr_clf]
labels = [clf.__class__.__name__ for clf in classifier_array]

normal_accuracy = []
normal_std = []
bagging_accuracy = []
bagging_std = []

for clf in classifier_array:
    cv_scores = cross_val_score(clf, X, y, cv=3, n_jobs=-1)
    
    normal_accuracy.append(np.round(cv_scores.mean(),4))
    normal_std.append(np.round(cv_scores.std(),4))
    
    print("Accuracy: %0.4f (+/- %0.4f) [Normal %s]" % (cv_scores.mean(), cv_scores.std(), clf.__class__.__name__))



Accuracy: 0.9213 (+/- 0.0424) [Normal RandomForestClassifier]
Accuracy: 0.6465 (+/- 0.0662) [Normal KNeighborsClassifier]
Accuracy: 0.9273 (+/- 0.0476) [Normal SVC]
Accuracy: 0.9108 (+/- 0.0925) [Normal LogisticRegression]


As we an see, our Random Forest, SVC, and Logistic Regression models performed reasonably well, but our K Nearest Neighbors model barely performed better than simple chance. Regardless, none of these models would be accurate enough to answer our business question. If these models were being used in the real world, we would not want our models to have too high of accuracies because that could indicate overfitting (future samples would not have accurate predictions because our model memorized this sample so specifically). However, this is a lab problem, and we have acces to the answers to our business problem. Due to this, we can aim for a 100% accuracy because in this case we truly are only using this model for a single sample set, so if our model memorizes this set, that actually works great. If we wanted to use these models in the future on other wines, we would have to take overfitting into consideration. In summary, I will be striving for 100% accuracy in this lab even though that is not usually a good approach in real life.

##Bagging
Below we use the bagging ensemble technique to improve our models from above using a maximum sample size of 0.4. Bagging creates subsets of the data by randomly sampling data points using replacement. Next, individual classifiers are trained using those subsets, and all of those predictions are combined to form the final predictions.

In [ ]:
for clf in classifier_array:
    cv_scores = cross_val_score(clf, X, y, cv=3, n_jobs=-1)
    bagging_clf = BaggingClassifier(clf, max_samples=0.4, max_features=3, random_state=RANDOM_SEED)
    bagging_scores = cross_val_score(bagging_clf, X, y, cv=3, n_jobs=-1)
    
    normal_accuracy.append(np.round(cv_scores.mean(),4))
    normal_std.append(np.round(cv_scores.std(),4))
    
    bagging_accuracy.append(np.round(bagging_scores.mean(),4))
    bagging_std.append(np.round(bagging_scores.std(),4))
    
    print("Accuracy: %0.4f (+/- %0.4f) [Normal %s]" % (cv_scores.mean(), cv_scores.std(), clf.__class__.__name__))
    print("Accuracy: %0.4f (+/- %0.4f) [Bagging %s]\n" % (bagging_scores.mean(), bagging_scores.std(), clf.__class__.__name__))

Accuracy: 0.9213 (+/- 0.0424) [Normal RandomForestClassifier]
Accuracy: 0.9440 (+/- 0.0205) [Bagging RandomForestClassifier]

Accuracy: 0.6465 (+/- 0.0662) [Normal KNeighborsClassifier]
Accuracy: 0.9215 (+/- 0.0442) [Bagging KNeighborsClassifier]

Accuracy: 0.9273 (+/- 0.0476) [Normal SVC]
Accuracy: 0.9441 (+/- 0.0417) [Bagging SVC]

Accuracy: 0.9108 (+/- 0.0925) [Normal LogisticRegression]
Accuracy: 0.9328 (+/- 0.0270) [Bagging LogisticRegression]



We can see from our output above that bagging improved the accuracy of all our models. The improvement to the K-Nearest Neighbors classifier was most substantial with about a 27% increase while the other models increased by around 2% each.

##Boosting
Below we use the boosting ensemble technique to improve our models. Boosting strengthens base learners by training them iteratively where each iteration tries to correct the error made in the previous iteration. We will use four forms of boosting in this lab: Ada Boost, Gradient Boost, XG Boost, and Ensemble. Ada Boost splits the variables into data stumps rather than data trees, which contain only one node and two leaves. Then, these stumps are trained and weighted based on accuracy until all the data points have been correctly classified or the maximum iteration level is reached (https://blog.paperspace.com/adaboost-optimizer/). Gradient Boost uses a loss function to minimize loss while adding decision trees one at a time (https://machinelearningmastery.com/gentle-introduction-gradient-boosting-algorithm-machine-learning/). XG Boost is a specific model defined within the scikit-learn library. It is basically a more advanced version of Gradient Boost using a convex loss function (https://machinelearningmastery.com/gentle-introduction-xgboost-applied-machine-learning/). Finally Ensemble or EnsembleVoteClassifier is a function from the MLEXTEND package that combines different model predictions using the concept of majority voting (https://medium.com/@saugata.paul1010/ensemble-learning-bagging-boosting-stacking-and-cascading-classifiers-in-machine-learning-9c66cb271674).

In [ ]:
ada_boost = AdaBoostClassifier(n_estimators=5)
grad_boost = GradientBoostingClassifier(n_estimators=10)
xgb_boost = XGBClassifier(max_depth=5, learning_rate=0.001)
ensemble_clf = EnsembleVoteClassifier(clfs=[ada_boost, grad_boost, xgb_boost], voting='hard')
boosting_labels = ['Ada Boost', 'Gradient Boost', 'XG Boost', 'Ensemble']
for clf, label in zip([ada_boost, grad_boost, xgb_boost, ensemble_clf], boosting_labels):
    scores = cross_val_score(clf, X, y, cv=3, scoring='accuracy')
    print("Accuracy: {0:.3f}, Variance: (+/-) {1:.3f} [{2}]".format(scores.mean(), scores.std(), label))

Accuracy: 0.843, Variance: (+/-) 0.035 [Ada Boost]
Accuracy: 0.837, Variance: (+/-) 0.063 [Gradient Boost]
Accuracy: 0.871, Variance: (+/-) 0.070 [XG Boost]
Accuracy: 0.871, Variance: (+/-) 0.053 [Ensemble]


We can see above that implementing boosting classifiers actually decreased our accuracy from the bagging classifiers. However, it these models still performed better than the k-Nearest Neighbors Base model.

##Stacked
Stacking combines separately trained models sequentially until a meta-classifier is created.

In [ ]:
RANDOM_SEED = 0
X = wine_dataframe.drop("target", 1)
y = wine_dataframe.target
from sklearn.preprocessing import LabelEncoder
encoder_object = LabelEncoder()
y = encoder_object.fit_transform(y)
#Base Learners
rf_clf = RandomForestClassifier(n_estimators=10, random_state=RANDOM_SEED)
knn_clf = KNeighborsClassifier(n_neighbors=2)
svc_clf = SVC(C=10000.0, kernel='rbf', random_state=RANDOM_SEED)
lr_clf = LogisticRegression(C=20000, penalty='l2', random_state=RANDOM_SEED)
lr = LogisticRegression(random_state=RANDOM_SEED) # meta classifier
lr_clf.max_iter = 100000
sclf = StackingClassifier(classifiers=[rf_clf, knn_clf, svc_clf, lr_clf], meta_classifier=lr)
classifier_array = [rf_clf, knn_clf, svc_clf, lr_clf, sclf ]
labels = [clf.__class__.__name__ for clf in classifier_array]
acc_list = []
var_list = []
for clf, label in zip(classifier_array, labels):
    cv_scores = model_selection.cross_val_score(clf, X, y, cv=3, scoring='accuracy')
    print("Accuracy: %0.4f (+/- %0.4f) [%s]" % (cv_scores.mean(), cv_scores.std(), label))
    acc_list.append(np.round(cv_scores.mean(),4))
    var_list.append(np.round(cv_scores.std(),4))\
    #print("Accuracy: %0.4f (+/- %0.4f) [%s]" % (cv_scores.mean(), cv_scores.std(), label))

Accuracy: 0.9046 (+/- 0.0442) [RandomForestClassifier]
Accuracy: 0.6465 (+/- 0.0662) [KNeighborsClassifier]
Accuracy: 0.9273 (+/- 0.0476) [SVC]
Accuracy: 0.9271 (+/- 0.0204) [LogisticRegression]
Accuracy: 0.9329 (+/- 0.0407) [StackingClassifier]


The final stacking model performed better than all of our previous models except Random Forest using Bagging and SVC using Bagging. Regardless, a 93% accuracy is fairly good especially considering one of the included models (k-Nearest Neighbors) had a initial accuracy of just 65%.

###Stacking with Grid Search Cross Validation
In the words of data scientist Satyam Kumar, "Grid Search Cross Validation tries all combinations of parameters grid for a model and returns with the best set of parameters having the best performance score" (https://towardsdatascience.com/20x-times-faster-grid-search-cross-validation-19ef01409b7c).

In [ ]:
#Base Learners.
rf_clf = RandomForestClassifier(random_state=RANDOM_SEED,n_jobs=-1)
knn_clf = KNeighborsClassifier(p=2, metric='minkowski',n_jobs=-1)
lr = LogisticRegression(random_state=RANDOM_SEED) # meta classifier
#sclf = StackingClassifier(classifiers=[rf_clf, et_clf, knn_clf, svc_clf, rg_clf, lr_clf, dt_clf, adab_clf], meta_classifier=lr)
sclf = StackingClassifier(classifiers=[rf_clf, knn_clf], meta_classifier=lr)
print("\nAccuracies of all classifiers using grid search cross validation.")
params = {'randomforestclassifier__n_estimators': np.arange(10,20), 'randomforestclassifier__max_depth': np.arange(1,5), 
          'kneighborsclassifier__n_neighbors': np.arange(1,20,2),
          'meta-logisticregression__C': [0.001,0.01,0.1,1,10,100,1000]}
gsearch_cv = GridSearchCV(estimator=sclf, param_grid=params, cv=5, refit=True)
gsearch_cv.fit(X, y)
cv_keys = ('mean_test_score', 'std_test_score', 'params')
print('Best parameters: %s' % gsearch_cv.best_params_)
print('Accuracy: %.2f' % gsearch_cv.best_score_)


Accuracies of all classifiers using grid search cross validation.
Best parameters: {'kneighborsclassifier__n_neighbors': 3, 'meta-logisticregression__C': 10, 'randomforestclassifier__max_depth': 4, 'randomforestclassifier__n_estimators': 15}
Accuracy: 0.98


Using Grid Search Cross Validation, our accuracy increased all the way to 98%. This is obviously the most accurate model, but it took 3 hours to compute for me which is the downside to it.

#Conclusion
After making predictions using different models with different ensemble learning techniques, we can see that Stacking with Grid Search Cross Validation is by far the most accurate method, and the basic k-Nearest Neighbors model is the worst method in terms of accuracy for this specific dataset. If we had the time to run Grid Search Cross Validation, that would be our best option for answering our business question accurately. If we were short on time, however, we might opt to use Random Forest using Bagging or SVC using Bagging because they are the most accurate of the other models and they do not take nearly as long as Grid Search Cross Validation. It makes sense that the k-Nearest Neighbors model is the worst because that model does not do well with unbalanced data. Even though this data is not that unbalanced, it is still too unbalanced for k-Nearest Neighbors to use accurately on its own. It also makes sense that Stacking with Grid Search Cross Validation was the most accurate because the algorithm spent an entire 3 hours testing every possible combination of parameters until it found the one that produced the highest accuracy. 